# connecting to google drive

In [11]:
# connecting google drive 

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
# changing working directory for easier access to data
%cd drive/My Drive/180/final project

/content/drive/My Drive/180/final project


# importing libraries

In [13]:
import pandas as pd 
import numpy as np 
import os
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

!pip install mrcnn
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

# testing image data

In [0]:
# from local data 
# path = '/Users/austinwilson/Desktop/CSUS/180/assignment3/project/vehicle-classification/backend/dump'
# test_file = '/Users/austinwilson/Desktop/CSUS/180/assignment3/project/vehicle-classification/backend/dump/sedan1.jpeg'

# for google drive 
gdrive_path = '/content/drive/My Drive/180/final project/dump'
gdrive_path2 = '/content/drive/My Drive/180/final project'

In [15]:
# testing to make sure images open properly
test_file = gdrive_path + '/coupe/coupe0.jpeg'
img = Image.open(test_file)
img

Output hidden; open in https://colab.research.google.com to view.

# DATA PREPROCESSING

In [0]:
# initializing empty arrays to store image data
coupe = []
coupe_y = []
sedan = []
sedan_y = []
motorcycle = []
motorcycle_y = []
img_size128 = (128,128)
img_size256 = (256,256)
img_size512 = (512,512)

In [17]:
# this may come in handy later if we decide to create new labels 
def get_class_labels():
    # short circuit if the folder w/ images DNE
    if not os.path.exists('dump'):
        raise Exception('dump DNE')
    if os.path.exists('dump/.DS_Store'):
      return os.listdir('dump')[1:]
    else:
      return os.listdir('dump')
classes = get_class_labels()
classes

['coupe', 'suv', 'truck', 'motorcycle', 'sedan']

## Importing images from local repository

In [0]:
def get_images_single_class(path,class_,image_size):
    full_path = path + os.path.sep + class_
    images = []
    classes = []
    img_size = image_size
    all_images = os.listdir(full_path)

    # so we can test a different number of images 
    num_images = len(all_images)
    for i in range(num_images):
        image_path = full_path + os.path.sep + all_images[i]
        if (not os.path.isfile(image_path)):
            continue
        try:
            img = Image.open(image_path)
            img = img.resize(img_size)
            img_array = np.asarray(img)
        except IOError as e:
            continue
        # had some issues with bad image size 
        # condition not working quite right so change this MANUALLY! 
        if (img_array.shape != (128,128,3) and 
            img_array.shape != (256,256,3) and
            img_array.shape != (512,512,3)):
          continue
        images.append(img_array)
        classes.append(class_)
    return np.asarray(images), np.asarray(classes)

## storing 128 and 256 pixel images:
 interested to see if this affects accuracy

In [0]:
# getting all the images and storing them as 128x128x3 pixel arrays
coupe128,y_coupe128 = get_images_single_class(gdrive_path,'coupe',img_size128)
motorcycle128,y_motorcycle128 = get_images_single_class(gdrive_path,'motorcycle',img_size128)
sedan128,y_sedan128 = get_images_single_class(gdrive_path,'sedan',img_size128)
suv128,y_suv128 = get_images_single_class(gdrive_path,'suv',img_size128)
truck128,y_truck128 = get_images_single_class(gdrive_path,'truck',img_size128)

In [0]:
# getting all the images and storing them as 256x256x3 pixel arrays
coupe256,y_coupe256 = get_images_single_class(gdrive_path,'coupe',img_size256)
motorcycle256,y_motorcycle256 = get_images_single_class(gdrive_path,'motorcycle',img_size256)
sedan256,y_sedan256 = get_images_single_class(gdrive_path,'sedan',img_size256)
suv256,y_suv256 = get_images_single_class(gdrive_path,'suv',img_size256)
truck256,y_truck256 = get_images_single_class(gdrive_path,'truck',img_size256)

In [21]:
# testing out image sizes 
print("x coupe: ",coupe128.shape,coupe256.shape,"\n")
print("y coupe",y_coupe128.shape,y_coupe256.shape,"\n")

print("x moto",motorcycle128.shape,motorcycle256.shape,"\n")
print("y moto",y_motorcycle128.shape,y_motorcycle256.shape,"\n")

print("x sedan",sedan128.shape,sedan256.shape,"\n")
print("y sedan",y_sedan128.shape,y_sedan256.shape,"\n")

print("x suv",suv128.shape,suv256.shape,"\n")
print("y suv",y_suv128.shape,y_suv256.shape,"\n")

print("x truck",truck128.shape,truck256.shape,"\n")
print("y truck",y_truck128.shape,y_truck256.shape,"\n")

x coupe:  (139, 128, 128, 3) (139, 256, 256, 3) 

y coupe (139,) (139,) 

x moto (124, 128, 128, 3) (124, 256, 256, 3) 

y moto (124,) (124,) 

x sedan (126, 128, 128, 3) (126, 256, 256, 3) 

y sedan (126,) (126,) 

x suv (140, 128, 128, 3) (140, 256, 256, 3) 

y suv (140,) (140,) 

x truck (139, 128, 128, 3) (139, 256, 256, 3) 

y truck (139,) (139,) 



In [0]:
# putting all x data into one array
X128 = np.concatenate([coupe128,motorcycle128,sedan128,suv128,truck128],axis=0)
X256 = np.concatenate([coupe256,motorcycle256,sedan256,suv256,truck256],axis=0)

In [0]:
# putting all y data into one array 
y128 = np.concatenate([y_coupe128,y_motorcycle128,y_sedan128,y_suv128,y_truck128],axis=0)
y256 = np.concatenate([y_coupe256,y_motorcycle256,y_sedan256,y_suv256,y_truck256],axis=0)
y = y256

In [24]:
# checking size of resulting arrays 
print(X128.shape,X256.shape,y128.shape,y256.shape)
print(y.shape)

(668, 128, 128, 3) (668, 256, 256, 3) (668,) (668,)
(668,)


Label encoding

In [0]:
le = LabelEncoder()
y128 = le.fit_transform(y128)
y256 = le.fit_transform(y256)
y = le.fit_transform(y)
y2 = le.fit_transform(y)

In [26]:
print(y.shape,y128.shape,y256.shape)
y = tf.keras.utils.to_categorical(y)
y128 = tf.keras.utils.to_categorical(y128)
y256 = tf.keras.utils.to_categorical(y256)
print(y.shape,y128.shape,y256.shape)

(668,) (668,) (668,)
(668, 5) (668, 5) (668, 5)


## train test split

In [0]:
x_train128,x_test128,y_train128,y_test128 = train_test_split(X128,y,test_size = .2, random_state = 42)
x_train256,x_test256,y_train256,y_test256 = train_test_split(X256,y,test_size = .2, random_state = 42)
# for resnet
x_train,x_test,y_train,y_test = train_test_split(X256,y2,test_size = .2, random_state = 42)

In [28]:
print(x_train128.shape,x_train256.shape)

print(y_train128.shape,y_train256.shape)

print(x_test128.shape,x_test256.shape)

print(y_test128.shape,y_test256.shape)
print(x_train128.shape[0] == y_train128.shape[0],x_train256.shape[0] == y_train256.shape[0])
print(x_test128.shape[0] == y_test128.shape[0],x_test256.shape[0] == y_test256.shape[0])

(534, 128, 128, 3) (534, 256, 256, 3)
(534, 5) (534, 5)
(134, 128, 128, 3) (134, 256, 256, 3)
(134, 5) (134, 5)
True True
True True


In [0]:
x_train128 = x_train128.astype('float32')
x_train256 = x_train256.astype('float32')

x_test128 = x_test128.astype('float32')
x_test256 = x_test256.astype('float32')

x_train128 /= 255
x_train256 /= 255

x_test128 /= 255
x_test256 /= 255

In [0]:
monitor = EarlyStopping(monitor='val_loss',min_delta=1e-3,patience=2,verbose=2,mode='auto')

# BASELINE MODEL: 
model with minimal layers as  the baseline

In [0]:
model_baseline128 = Sequential()
model_baseline128.add(Dense(32,input_shape=(128,128,3),activation='relu'))
model_baseline128.add(Flatten())
model_baseline128.add(Dense(5,activation='softmax'))
model_baseline128.compile(loss='categorical_crossentropy',optimizer='adam')

In [33]:
model_baseline128.fit(x_train128,y_train128,
                   batch_size = 20,
                   epochs = 20,
                   verbose=2,
                   callbacks=[monitor],
                   validation_data=(x_test128,y_test128))

Epoch 1/20
27/27 - 1s - loss: 11.8370 - val_loss: 3.7783
Epoch 2/20
27/27 - 1s - loss: 2.1426 - val_loss: 3.1333
Epoch 3/20
27/27 - 1s - loss: 1.2160 - val_loss: 1.9707
Epoch 4/20
27/27 - 1s - loss: 0.7182 - val_loss: 1.8002
Epoch 5/20
27/27 - 1s - loss: 0.4319 - val_loss: 1.8455
Epoch 6/20
27/27 - 1s - loss: 0.2674 - val_loss: 2.2093
Epoch 00006: early stopping


## baseline for 128 pixel images

In [34]:
y_true_baseline128 = np.argmax(y_test128,axis=1)
baseline_pred128 = model_baseline128.predict(x_test128)
baseline_pred128 = np.argmax(baseline_pred128,axis=1)
f1_baseline128 = metrics.f1_score(y_true_baseline128,baseline_pred128,average='weighted')
accuracy_baseline128 = metrics.accuracy_score(y_true_baseline128,baseline_pred128)
recall_baseline128 = metrics.recall_score(y_true_baseline128,baseline_pred128,average = 'weighted')
print('F1: {}\nAccuracy: {}\nRecall: {}'.format(f1_baseline128,accuracy_baseline128,recall_baseline128))

F1: 0.3387949942537959
Accuracy: 0.3656716417910448
Recall: 0.3656716417910448


In [37]:
print(metrics.classification_report(y_true_baseline128,baseline_pred128))

              precision    recall  f1-score   support

           0       0.54      0.18      0.27        39
           1       0.45      0.25      0.32        20
           2       0.50      0.19      0.28        26
           3       0.26      0.58      0.35        19
           4       0.37      0.70      0.48        30

    accuracy                           0.37       134
   macro avg       0.42      0.38      0.34       134
weighted avg       0.44      0.37      0.34       134



In [0]:
model_baseline = Sequential()
model_baseline.add(Dense(32,input_shape=(256,256,3),activation='relu'))
model_baseline.add(Flatten())
model_baseline.add(Dense(5,activation='softmax'))
model_baseline.compile(loss='categorical_crossentropy',optimizer='adam')

In [39]:
model_baseline.fit(x_train256,y_train256,
                   batch_size = 20,
                   epochs = 20,
                   verbose=2,
                   callbacks=[monitor],
                   validation_data=(x_test256,y_test256))

Epoch 1/20
27/27 - 2s - loss: 38.4086 - val_loss: 16.5042
Epoch 2/20
27/27 - 2s - loss: 5.7757 - val_loss: 9.5438
Epoch 3/20
27/27 - 2s - loss: 2.9649 - val_loss: 5.2408
Epoch 4/20
27/27 - 2s - loss: 1.9443 - val_loss: 4.8995
Epoch 5/20
27/27 - 2s - loss: 0.8923 - val_loss: 4.4221
Epoch 6/20
27/27 - 2s - loss: 0.7053 - val_loss: 4.5869
Epoch 7/20
27/27 - 2s - loss: 0.3948 - val_loss: 4.5856
Epoch 00007: early stopping


## baseline for 256 pixel predictions 

In [42]:
y_true_baseline256 = np.argmax(y_test256,axis=1)
baseline_pred = model_baseline.predict(x_test256)
baseline_pred = np.argmax(baseline_pred,axis=1)
f1_baseline = metrics.f1_score(y_true_baseline,baseline_pred,average='weighted')
accuracy_baseline = metrics.accuracy_score(y_true_baseline,baseline_pred)
recall_baseline = metrics.recall_score(y_true_baseline,baseline_pred,average = 'weighted')
print('F1: {}\nAccuracy: {}\nRecall: {}'.format(f1_baseline,accuracy_baseline,recall_baseline))

F1: 0.387580485363956
Accuracy: 0.41044776119402987
Recall: 0.41044776119402987


In [43]:
print(metrics.classification_report(y_true_baseline256,baseline_pred))

              precision    recall  f1-score   support

           0       0.36      0.26      0.30        39
           1       0.60      0.15      0.24        20
           2       0.45      0.38      0.42        26
           3       0.34      0.63      0.44        19
           4       0.45      0.67      0.54        30

    accuracy                           0.41       134
   macro avg       0.44      0.42      0.39       134
weighted avg       0.43      0.41      0.39       134



In [0]:
input_shape128 = (128,128,3)
input_shape256 = (256,256,3)
input_shape512 = (512,512,3)

# DEEP NEURAL NETWORK:
model from cifar10 without transfer learning

In [0]:
def get_modelo(input_shape):

  activator = 'relu'
  optimizer = 'adam'


  modelo_ar = Sequential()
  modelo_ar.add(Conv2D(32, (3, 3), activation=activator, padding='same', input_shape=input_shape))
  modelo_ar.add(Conv2D(32, (3, 3), activation=activator, padding='same'))
  modelo_ar.add(Conv2D(32, (3, 3), activation=activator, padding='same'))
  modelo_ar.add(MaxPooling2D((2, 2)))
  modelo_ar.add(Dropout(0.2))
  modelo_ar.add(Conv2D(64, (3, 3), activation=activator, padding='same'))
  modelo_ar.add(Conv2D(64, (3, 3), activation=activator, padding='same'))
  modelo_ar.add(Conv2D(64, (3, 3), activation=activator, padding='same'))
  modelo_ar.add(MaxPooling2D((2, 2)))
  modelo_ar.add(Dropout(0.2))
  modelo_ar.add(Conv2D(128, (3, 3), activation=activator, padding='same'))
  modelo_ar.add(Conv2D(128, (3, 3), activation=activator, padding='same'))
  modelo_ar.add(MaxPooling2D((2, 2)))
  modelo_ar.add(Flatten())
  modelo_ar.add(Dense(128, activation=activator))
  modelo_ar.add(Dropout(0.2))
  modelo_ar.add(Dense(5, activation='softmax'))
  modelo_ar.compile(loss='categorical_crossentropy',optimizer=optimizer)
  return modelo_ar

In [0]:
modelo128 = get_modelo(input_shape128)
modelo256 = get_modelo(input_shape256)

In [47]:
modelo128.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 64)       

## training model

In [0]:
monitor = EarlyStopping(monitor='val_loss',min_delta=1e-3,patience=2,verbose=2,mode='auto')
checkpointer128 = ModelCheckpoint(filepath='/content/drive/My Drive/180/final project/dnn/modelo128.hdf5',verbose=0,save_best_only=True)
checkpointer256 = ModelCheckpoint(filepath='/content/drive/My Drive/180/final project/dnn/modelo256.hdf5',verbose=0,save_best_only=True)

In [49]:
modelo128.fit(x_train128,y_train128,
    batch_size=10,
    epochs=10,
    verbose=2,
    validation_data=(x_test128,y_test128),
    callbacks=[monitor,checkpointer128]
    )

Epoch 1/10
54/54 - 4s - loss: 1.6202 - val_loss: 1.6110
Epoch 2/10
54/54 - 3s - loss: 1.6092 - val_loss: 1.6109
Epoch 3/10
54/54 - 3s - loss: 1.6088 - val_loss: 1.6124
Epoch 00003: early stopping


In [50]:
modelo256.fit(x_train256,y_train256,
    batch_size=10,
    epochs=10,
    verbose=2,
    validation_data=(x_test256,y_test256),
    callbacks=[monitor,checkpointer256]
    )

Epoch 1/10
54/54 - 12s - loss: 1.6223 - val_loss: 1.6101
Epoch 2/10
54/54 - 9s - loss: 1.6092 - val_loss: 1.6106
Epoch 3/10
54/54 - 9s - loss: 1.6088 - val_loss: 1.6128
Epoch 00003: early stopping


In [0]:
from sklearn import metrics
y_true128 = np.argmax(y_test128,axis=1)
y_true256 = np.argmax(y_test256,axis=1)

In [0]:
pred_modelo128 = modelo128.predict(x_test128)
pred_modelo256 = modelo256.predict(x_test256)
pred_modelo128 = np.argmax(pred_modelo128,axis=1)
pred_modelo256 = np.argmax(pred_modelo256,axis=1)

In [0]:
f1_modelo128 = metrics.f1_score(y_true128,pred_modelo128,average='weighted')
recall_modelo128 = metrics.recall_score(y_true128,pred_modelo128,average='weighted')
accuracy_model128 = metrics.accuracy_score(y_true128,pred_modelo128)

f1_modelo256 = metrics.f1_score(y_true256,pred_modelo256,average='weighted')
recall_modelo256 = metrics.recall_score(y_true256,pred_modelo256,average='weighted')
accuracy_model256 = metrics.accuracy_score(y_true256,pred_modelo256)

In [54]:
print("modelo 128")
print("F1: {}\nRecall:{}\nAccuracy:{}".format(f1_modelo128,recall_modelo128,accuracy_model128))

print("model 256")
print("F1: {}\nRecall:{}\nAccuracy:{}".format(f1_modelo256,recall_modelo256,accuracy_model256))

modelo 128
F1: 0.035216076480343375
Recall:0.1417910447761194
Accuracy:0.1417910447761194
model 256
F1: 0.035216076480343375
Recall:0.1417910447761194
Accuracy:0.1417910447761194


In [55]:
print(metrics.classification_report(y_true128,pred_modelo128))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        26
           3       0.14      1.00      0.25        19
           4       0.00      0.00      0.00        30

    accuracy                           0.14       134
   macro avg       0.03      0.20      0.05       134
weighted avg       0.02      0.14      0.04       134



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
print(metrics.classification_report(y_true256,pred_modelo256))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        26
           3       0.14      1.00      0.25        19
           4       0.00      0.00      0.00        30

    accuracy                           0.14       134
   macro avg       0.03      0.20      0.05       134
weighted avg       0.02      0.14      0.04       134



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# TRANSFER LEARNING:
VGG16, VGG19, RESNET

In [0]:
monitor = EarlyStopping(monitor='val_loss',min_delta=1e-3,patience=2,verbose=2,mode='auto')
checkpointer128 = ModelCheckpoint(filepath='/content/drive/My Drive/180/final project/dnn/weights_model128.hdf5',verbose=0,save_best_only=True)
checkpointer256 = ModelCheckpoint(filepath='/content/drive/My Drive/180/final project/dnn/weights_model256.hdf5',verbose=0,save_best_only=True)

### VGG16

In [58]:
vgg_model128 = VGG16(weights='imagenet', include_top=False, input_shape=(128,128, 3))
vgg_model256 = VGG16(weights='imagenet', include_top=False, input_shape=(256,256, 3)) 

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
# 128 px model
model128 = Sequential()
for layer in vgg_model128.layers:
  model128.add(layer)
for layer in model128.layers:
  layer.trainable = False

In [0]:
# 256 model
model256 = Sequential()
for layer in vgg_model256.layers:
  model256.add(layer)
for layer in model256.layers:
  layer.trainable = False

In [0]:
model128.add(Flatten())
model128.add(Dense(5,activation='softmax'))

In [0]:
model256.add(Flatten())
model256.add(Dense(5,activation='softmax'))

In [0]:
model128.compile(loss='categorical_crossentropy',optimizer='adam')
model256.compile(loss='categorical_crossentropy',optimizer='adam')

In [64]:
for i in range(5):
  model128.fit(x_train128,y_train128,
              batch_size=128,
              epochs=10,
              verbose=2,
              validation_data=(x_test128,y_test128),
              callbacks=[monitor,checkpointer128])

Epoch 1/10
5/5 - 5s - loss: 1.5030 - val_loss: 1.1771
Epoch 2/10
5/5 - 2s - loss: 0.8987 - val_loss: 0.7586
Epoch 3/10
5/5 - 2s - loss: 0.5881 - val_loss: 0.6714
Epoch 4/10
5/5 - 2s - loss: 0.4752 - val_loss: 0.5440
Epoch 5/10
5/5 - 2s - loss: 0.4062 - val_loss: 0.5190
Epoch 6/10
5/5 - 2s - loss: 0.3622 - val_loss: 0.4971
Epoch 7/10
5/5 - 2s - loss: 0.2973 - val_loss: 0.4760
Epoch 8/10
5/5 - 2s - loss: 0.2990 - val_loss: 0.4694
Epoch 9/10
5/5 - 2s - loss: 0.2571 - val_loss: 0.4572
Epoch 10/10
5/5 - 2s - loss: 0.2609 - val_loss: 0.4601
Epoch 1/10
5/5 - 2s - loss: 0.2008 - val_loss: 0.4279
Epoch 2/10
5/5 - 2s - loss: 0.1947 - val_loss: 0.4696
Epoch 3/10
5/5 - 2s - loss: 0.1817 - val_loss: 0.4419
Epoch 00003: early stopping
Epoch 1/10
5/5 - 2s - loss: 0.1617 - val_loss: 0.4397
Epoch 2/10
5/5 - 2s - loss: 0.1596 - val_loss: 0.4626
Epoch 3/10
5/5 - 2s - loss: 0.1531 - val_loss: 0.4388
Epoch 00003: early stopping
Epoch 1/10
5/5 - 2s - loss: 0.1362 - val_loss: 0.4707
Epoch 2/10
5/5 - 2s - los

In [65]:
for i in range(5):
  print(i)
  model256.fit(x_train256,y_train256,
              batch_size=128,
              epochs=10,
              verbose=2,
              validation_data=(x_test256,y_test256),
              callbacks=[monitor,checkpointer256])

0
Epoch 1/10
5/5 - 10s - loss: 2.4929 - val_loss: 1.8293
Epoch 2/10
5/5 - 6s - loss: 1.1608 - val_loss: 1.0286
Epoch 3/10
5/5 - 6s - loss: 0.7001 - val_loss: 0.5442
Epoch 4/10
5/5 - 6s - loss: 0.3761 - val_loss: 0.4388
Epoch 5/10
5/5 - 6s - loss: 0.2460 - val_loss: 0.4605
Epoch 6/10
5/5 - 7s - loss: 0.2141 - val_loss: 0.3541
Epoch 7/10
5/5 - 6s - loss: 0.1232 - val_loss: 0.3575
Epoch 8/10
5/5 - 7s - loss: 0.0841 - val_loss: 0.2951
Epoch 9/10
5/5 - 6s - loss: 0.0613 - val_loss: 0.2976
Epoch 10/10
5/5 - 6s - loss: 0.0421 - val_loss: 0.3087
Epoch 00010: early stopping
1
Epoch 1/10
5/5 - 7s - loss: 0.0361 - val_loss: 0.2776
Epoch 2/10
5/5 - 6s - loss: 0.0283 - val_loss: 0.2809
Epoch 3/10
5/5 - 6s - loss: 0.0259 - val_loss: 0.2928
Epoch 00003: early stopping
2
Epoch 1/10
5/5 - 6s - loss: 0.0232 - val_loss: 0.2806
Epoch 2/10
5/5 - 7s - loss: 0.0226 - val_loss: 0.2759
Epoch 3/10
5/5 - 6s - loss: 0.0193 - val_loss: 0.2798
Epoch 4/10
5/5 - 6s - loss: 0.0180 - val_loss: 0.2895
Epoch 00004: early

In [0]:
pred128 = model128.predict(x_test128)
pred128 = np.argmax(pred128,axis=1)

In [0]:
pred256 = model256.predict(x_test256)
pred256 = np.argmax(pred256,axis=1)

In [0]:
y_true128 = np.argmax(y_test128,axis=1)
y_true256 = np.argmax(y_test256,axis=1)
y_true = y_true256

In [70]:
f1128 = metrics.f1_score(y_true128,pred128,average='weighted')
recall128 = metrics.recall_score(y_true128,pred128,average='weighted')
accuracy128 = metrics.accuracy_score(y_true128,pred128)

f1256 = metrics.f1_score(y_true256,pred256,average='weighted')
accuracy256 = metrics.accuracy_score(y_true256,pred256)
recall256 = metrics.recall_score(y_true,pred256,average='weighted')
print("128\nF1: {}\nRecall: {}\nAccuracy:{}".format(f1128,recall128,accuracy128))
print("256\nF1: {}\nRecall: {}\nAccuracy:{}".format(f1256,recall256,accuracy256))
# print("F1 256x256 image :",f1256)
# print("Accuracy: ",accuracy)
# print("Recall: ",recall)

128
F1: 0.7506558118498418
Recall: 0.753731343283582
Accuracy:0.753731343283582
256
F1: 0.7606247179555748
Recall: 0.7611940298507462
Accuracy:0.7611940298507462


In [71]:
print(metrics.classification_report(y_true128,pred128))

              precision    recall  f1-score   support

           0       0.70      0.49      0.58        39
           1       1.00      1.00      1.00        20
           2       0.56      0.73      0.63        26
           3       0.61      0.74      0.67        19
           4       0.97      0.97      0.97        30

    accuracy                           0.75       134
   macro avg       0.77      0.78      0.77       134
weighted avg       0.77      0.75      0.75       134



In [72]:
print(metrics.classification_report(y_true256,pred256))

              precision    recall  f1-score   support

           0       0.80      0.51      0.62        39
           1       1.00      1.00      1.00        20
           2       0.58      0.69      0.63        26
           3       0.54      0.79      0.64        19
           4       0.97      0.97      0.97        30

    accuracy                           0.76       134
   macro avg       0.78      0.79      0.77       134
weighted avg       0.79      0.76      0.76       134



## MOBILE DEPLOYMENT:
testing saving/loading weights and model for the classification script

In [0]:
# saving model for classification script
# model128.save(filepath = '/content/drive/My Drive/180/final project/dnn/model128_final.hdf5')
# model256.save(filepath = '/content/drive/My Drive/180/final project/dnn/model256_final.hdf5')
# test128 = tf.keras.models.load_model('/content/drive/My Drive/180/final project/dnn/model128_final.hdf5')
# test256 = tf.keras.models.load_model('/content/drive/My Drive/180/final project/dnn/model256_final.hdf5')
# test128.load_weights('/content/drive/My Drive/180/final project/dnn/weights_model128.hdf5')
# test256.load_weights('/content/drive/My Drive/180/final project/dnn/weights_model256.hdf5')
# test128.compile(loss='categorical_crossentropy',optimizer='adam')
# test256.compile(loss='categorical_crossentropy',optimizer='adam')
# test128_pred = test128.predict(x_test128)
# test128_pred = np.argmax(test128_pred,axis=1)
# test256_pred = test256.predict(x_test256)
# test256_pred =np.argmax(test256_pred,axis=1)
# test_f1128 = metrics.f1_score(y_true128,test128_pred,average='weighted')
# test_f1256 = metrics.f1_score(y_true256,test256_pred,average='weighted')
# print(test_f1128)
# print(test_f1256)
# test_model.load_weights('/content/drive/My Drive/180/final project/dnn/weights_model2.hdf5')
# test_model.compile(loss='categorical_crossentropy',optimizer='adam')
# pred_test = test_model.predict(x_test256)
# pred_test = np.argmax(pred_test,axis=1)
# y_true = np.argmax(y_test256,axis=1)
# test_f1 = metrics.f1_score(y_true,pred_test,average='weighted')
# print('F1:',test_f1)

## VGG19

In [73]:
vgg19_128 = VGG19(weights='imagenet',include_top=False,input_shape=(128,128,3))
vgg19_256 = VGG19(weights='imagenet',include_top=False,input_shape=(256,256,3))

80142336/80134624 [==============================] - 1s 0us/step


In [0]:
model_19_128 = Sequential()
for layer in vgg19_128.layers:
  model_19_128.add(layer)
for layer in model_19_128.layers:
  layer.trainable = False

In [0]:
model_19_256 = Sequential()
for layer in vgg19_256.layers:
  model_19_256.add(layer)
for layer in model_19_256.layers:
  layer.trainable = False

In [0]:
model_19_128.add(Flatten())
model_19_128.add(Dense(5,activation='softmax'))

In [0]:
model_19_256.add(Flatten())
model_19_256.add(Dense(5,activation='softmax'))

In [0]:
checkpointer_vgg19_128 = ModelCheckpoint(filepath='/content/drive/My Drive/180/final project/dnn/weights_model_vgg19_128.hdf5',verbose=0,save_best_only=True)
model_19_128.compile(optimizer='adam',loss='categorical_crossentropy')

In [0]:
checkpointer_vgg19_256 = ModelCheckpoint(filepath='/content/drive/My Drive/180/final project/dnn/weights_model_vgg19_256.hdf5',verbose=0,save_best_only=True)
model_19_256.compile(optimizer='adam',loss='categorical_crossentropy')

In [83]:
for i in range(5):
  model_19_128.fit(x_train128,y_train128,
              batch_size=32,
              epochs=20,
              verbose=2,
              callbacks=[monitor,checkpointer_vgg19_128],
              validation_data=(x_test128,y_test128))

Epoch 1/20
17/17 - 3s - loss: 1.3084 - val_loss: 1.0303
Epoch 2/20
17/17 - 3s - loss: 0.5899 - val_loss: 0.7515
Epoch 3/20
17/17 - 3s - loss: 0.4021 - val_loss: 0.6523
Epoch 4/20
17/17 - 3s - loss: 0.3207 - val_loss: 0.6371
Epoch 5/20
17/17 - 3s - loss: 0.2534 - val_loss: 0.5573
Epoch 6/20
17/17 - 3s - loss: 0.2034 - val_loss: 0.5402
Epoch 7/20
17/17 - 2s - loss: 0.1701 - val_loss: 0.5668
Epoch 8/20
17/17 - 2s - loss: 0.1425 - val_loss: 0.5746
Epoch 00008: early stopping
Epoch 1/20
17/17 - 3s - loss: 0.1299 - val_loss: 0.5286
Epoch 2/20
17/17 - 2s - loss: 0.1040 - val_loss: 0.5983
Epoch 3/20
17/17 - 2s - loss: 0.0931 - val_loss: 0.5681
Epoch 00003: early stopping
Epoch 1/20
17/17 - 2s - loss: 0.0809 - val_loss: 0.5524
Epoch 2/20
17/17 - 2s - loss: 0.0728 - val_loss: 0.5735
Epoch 3/20
17/17 - 2s - loss: 0.0655 - val_loss: 0.5733
Epoch 00003: early stopping
Epoch 1/20
17/17 - 2s - loss: 0.0592 - val_loss: 0.5637
Epoch 2/20
17/17 - 2s - loss: 0.0531 - val_loss: 0.6039
Epoch 3/20
17/17 - 2

In [84]:
for i in range(5):
  model_19_256.fit(x_train256,y_train256,
              batch_size=32,
              epochs=20,
              verbose=2,
              callbacks=[monitor,checkpointer_vgg19_256],
              validation_data=(x_test256,y_test256))

Epoch 1/20
17/17 - 9s - loss: 1.9161 - val_loss: 1.2685
Epoch 2/20
17/17 - 9s - loss: 0.4699 - val_loss: 0.6047
Epoch 3/20
17/17 - 9s - loss: 0.1637 - val_loss: 0.3855
Epoch 4/20
17/17 - 8s - loss: 0.0630 - val_loss: 0.4419
Epoch 5/20
17/17 - 9s - loss: 0.0414 - val_loss: 0.3823
Epoch 6/20
17/17 - 8s - loss: 0.0272 - val_loss: 0.4373
Epoch 7/20
17/17 - 8s - loss: 0.0216 - val_loss: 0.3983
Epoch 00007: early stopping
Epoch 1/20
17/17 - 8s - loss: 0.0180 - val_loss: 0.4206
Epoch 2/20
17/17 - 8s - loss: 0.0152 - val_loss: 0.4048
Epoch 3/20
17/17 - 8s - loss: 0.0136 - val_loss: 0.4043
Epoch 4/20
17/17 - 8s - loss: 0.0124 - val_loss: 0.4052
Epoch 00004: early stopping
Epoch 1/20
17/17 - 8s - loss: 0.0104 - val_loss: 0.3964
Epoch 2/20
17/17 - 8s - loss: 0.0093 - val_loss: 0.4175
Epoch 3/20
17/17 - 8s - loss: 0.0084 - val_loss: 0.4144
Epoch 00003: early stopping
Epoch 1/20
17/17 - 8s - loss: 0.0076 - val_loss: 0.4129
Epoch 2/20
17/17 - 8s - loss: 0.0071 - val_loss: 0.4017
Epoch 3/20
17/17 - 8

In [0]:
pred19_128 = model_19_128.predict(x_test128)
pred19_128 = np.argmax(pred19_128,axis=1)

pred19_256 = model_19_256.predict(x_test256)
pred19_256 = np.argmax(pred19_256,axis=1)

In [88]:
f1_vgg19_128 = metrics.f1_score(y_true128,pred19_128,average='weighted')
accuracy_vgg19_128 = metrics.accuracy_score(y_true128,pred19_128)
recall_vgg19_128 = metrics.recall_score(y_true128,pred19_128,average='weighted')

f1_vgg19_256 = metrics.f1_score(y_true256,pred19_256,average='weighted')
accuracy_vgg19_256 = metrics.accuracy_score(y_true256,pred19_256)
recall_vgg19_256 = metrics.recall_score(y_true256,pred19_256,average='weighted')


print('VGG19 128\nF1: {}\nAccuracy: {}\nRecall: {}'.format(f1_vgg19_128,accuracy_vgg19_128,recall_vgg19_128))
print()
print('VGG19 256\nF1: {}\nAccuracy: {}\nRecall: {}'.format(f1_vgg19_256,accuracy_vgg19_256,recall_vgg19_256))

VGG19 128
F1: 0.7412357205613765
Accuracy: 0.7388059701492538
Recall: 0.7388059701492538

VGG19 256
F1: 0.8182376726733371
Accuracy: 0.8208955223880597
Recall: 0.8208955223880597


In [91]:
print(metrics.classification_report(y_true128,pred19_128))

              precision    recall  f1-score   support

           0       0.73      0.56      0.64        39
           1       0.95      0.95      0.95        20
           2       0.63      0.65      0.64        26
           3       0.50      0.74      0.60        19
           4       0.93      0.90      0.92        30

    accuracy                           0.74       134
   macro avg       0.75      0.76      0.75       134
weighted avg       0.76      0.74      0.74       134



In [92]:
print(metrics.classification_report(y_true256,pred19_256))

              precision    recall  f1-score   support

           0       0.82      0.59      0.69        39
           1       1.00      1.00      1.00        20
           2       0.70      0.73      0.72        26
           3       0.63      1.00      0.78        19
           4       1.00      0.97      0.98        30

    accuracy                           0.82       134
   macro avg       0.83      0.86      0.83       134
weighted avg       0.84      0.82      0.82       134



## RESNET TAKE 3

In [0]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   zoom_range=0.3, 
                                   rotation_range=50,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2,
                                   horizontal_flip=True, 
                                   fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow(x_train, y_train,batch_size=30)
val_generator = val_datagen.flow(x_test, y_test, batch_size=30)

In [0]:
img_size = (256,256,3)

In [97]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=img_size)
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
# restnet.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 4s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  import sys


In [0]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu', input_dim=img_size))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
# model.summary()

In [0]:
checkpointer_resnet = ModelCheckpoint(filepath='/content/drive/My Drive/180/final project/dnn/weights_model_resnet.hdf5',verbose=0,save_best_only=True)

In [100]:
history = model.fit_generator(train_generator, 
                              steps_per_epoch=10, 
                              epochs=20,
                              validation_data=val_generator, 
                              validation_steps=50, 
                              verbose=1)

Epoch 1/20
10/10 [==============================] - 31s 3s/step - loss: -48.6385 - accuracy: 0.1833 - val_loss: -9.8129 - val_accuracy: 0.1493
Epoch 2/20
10/10 [==============================] - 24s 2s/step - loss: -139.8707 - accuracy: 0.1939 - val_loss: -19.2262 - val_accuracy: 0.1493
Epoch 3/20
10/10 [==============================] - 23s 2s/step - loss: -209.5414 - accuracy: 0.2041 - val_loss: -76.2874 - val_accuracy: 0.1493
Epoch 4/20
10/10 [==============================] - 24s 2s/step - loss: -341.1104 - accuracy: 0.1967 - val_loss: -157.4906 - val_accuracy: 0.1493
Epoch 5/20
10/10 [==============================] - 23s 2s/step - loss: -420.7772 - accuracy: 0.2041 - val_loss: -119.2515 - val_accuracy: 0.1493
Epoch 6/20
10/10 [==============================] - 24s 2s/step - loss: -556.0577 - accuracy: 0.1599 - val_loss: -121.2541 - val_accuracy: 0.1493
Epoch 7/20
10/10 [==============================] - 23s 2s/step - loss: -670.4948 - accuracy: 0.2200 - val_loss: -181.8242 - val_

In [0]:
pred_rn = model.predict(x_test256)

In [0]:
pred_rn = np.argmax(pred_rn,axis=1)

In [104]:
f1_rn = metrics.f1_score(y_true256,pred_rn,average='weighted')
accuracy_rn = metrics.accuracy_score(y_true256,pred_rn)
recall_rn = metrics.recall_score(y_true256,pred_rn,average='weighted')
print("F1: {}\nAccuracy: {}\nRecall: {}".format(f1_rn,accuracy_rn,recall_rn))

F1: 0.1312225002156846
Accuracy: 0.291044776119403
Recall: 0.291044776119403
